In [43]:
import pandas as pd
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

In [44]:
data=pd.read_csv('survey_results_public.csv')
data.head()

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0
3,4,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,Automated testing;Continuous integration (CI) ...,NaN,Appropriate in length,Easy,156000.0
4,5,I agree,I am a developer by profession,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,Microservices;Automated testing;Observability ...,Other,Appropriate in length,Neither easy nor difficult,23456.0


In [45]:
data.drop(data[data['ConvertedCompYearly'] > 299999].index, inplace=True)
data.drop(data[data['ConvertedCompYearly'] < 12999].index, inplace=True)

## Data Wrangling

### Looking the data

In [46]:
data.shape

(84138, 84)

In [47]:
data.columns

Index(['ResponseId', 'Q120', 'MainBranch', 'Age', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool', 'Country',
       'Currency', 'CompTotal', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use',
       'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith',
       'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncHaveWorkedWith',
       'OfficeStackSyncWantToWorkWith', 'AISearchHaveWorkedWith',
       'AISearchWan

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84138 entries, 0 to 89183
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           84138 non-null  int64  
 1   Q120                                 84138 non-null  object 
 2   MainBranch                           84138 non-null  object 
 3   Age                                  84138 non-null  object 
 4   Employment                           82855 non-null  object 
 5   RemoteWork                           68786 non-null  object 
 6   CodingActivities                     68743 non-null  object 
 7   EdLevel                              82927 non-null  object 
 8   LearnCode                            82625 non-null  object 
 9   LearnCodeOnline                      66136 non-null  object 
 10  LearnCodeCoursesCert                 34542 non-null  object 
 11  YearsCode                        

In [49]:
# data.drop(data['ConvertedCompYearly'].nlargest(16).index, inplace=True)

In [50]:
data.dropna(subset=['ConvertedCompYearly'], inplace=True)


In [51]:
data['ConvertedCompYearly'].isna().sum()

0

In [52]:
data['DevType'].value_counts()

DevType
Developer, full-stack                            15317
Developer, back-end                               8567
Developer, front-end                              2780
Developer, desktop or enterprise applications     2242
Developer, mobile                                 1381
Other (please specify):                           1339
Engineering manager                               1208
Developer, embedded applications or devices       1181
DevOps specialist                                  914
Data scientist or machine learning specialist      882
Engineer, data                                     834
Research & Development role                        765
Cloud infrastructure engineer                      705
Senior Executive (C-Suite, VP, etc.)               669
Academic researcher                                560
Developer, game or graphics                        441
Data or business analyst                           409
Developer, QA or test                              311
Sy

## First Separation (Only what is needed), If it were only for data

In [53]:
only_data = data.copy()#[data['DevType'].isin(['Data or business analyst', 'Engineer, data', 'Data scientist or machine learning specialist'])].copy()
only_data['DevType'].value_counts()

DevType
Developer, full-stack                            15317
Developer, back-end                               8567
Developer, front-end                              2780
Developer, desktop or enterprise applications     2242
Developer, mobile                                 1381
Other (please specify):                           1339
Engineering manager                               1208
Developer, embedded applications or devices       1181
DevOps specialist                                  914
Data scientist or machine learning specialist      882
Engineer, data                                     834
Research & Development role                        765
Cloud infrastructure engineer                      705
Senior Executive (C-Suite, VP, etc.)               669
Academic researcher                                560
Developer, game or graphics                        441
Data or business analyst                           409
Developer, QA or test                              311
Sy

In [54]:
only_data.dtypes

ResponseId               int64
Q120                    object
MainBranch              object
Age                     object
Employment              object
                        ...   
ProfessionalTech        object
Industry                object
SurveyLength            object
SurveyEase              object
ConvertedCompYearly    float64
Length: 84, dtype: object

In [55]:
only_data.columns

Index(['ResponseId', 'Q120', 'MainBranch', 'Age', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool', 'Country',
       'Currency', 'CompTotal', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use',
       'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith',
       'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncHaveWorkedWith',
       'OfficeStackSyncWantToWorkWith', 'AISearchHaveWorkedWith',
       'AISearchWan

## The columns we want

In [56]:
data_test = only_data[['RemoteWork','EdLevel','DevType','OrgSize','Country','ConvertedCompYearly','YearsCodePro','LanguageHaveWorkedWith']].copy()#,

### Country

In [57]:
for i in data_test['Country']:
    if data_test['Country'].value_counts()[i]<200:
        data_test['Country'].replace(i,"Other",inplace=True)


In [58]:
data_test[data_test['Country']=='Portugal']

,RemoteWork,EdLevel,DevType,OrgSize,Country,ConvertedCompYearly,YearsCodePro,LanguageHaveWorkedWith
33,Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Developer, front-end","1,000 to 4,999 employees",Portugal,74963.0,5,TypeScript
154,Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Developer Experience,"10,000 or more employees",Portugal,69608.0,12,C++;Go;JavaScript;Python;TypeScript
337,"Hybrid (some remote, some in-person)",Some college/university study without earning ...,"Developer, back-end",10 to 19 employees,Portugal,25701.0,4,C#;HTML/CSS;JavaScript;Python;SQL
591,Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Developer, back-end",500 to 999 employees,Portugal,107090.0,17,C#;HTML/CSS
659,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, full-stack","10,000 or more employees",Portugal,34269.0,25,JavaScript;PHP
...,...,...,...,...,...,...,...,...
87403,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, QA or test",100 to 499 employees,Portugal,29985.0,5,Groovy;TypeScript
87797,Remote,Some college/university study without earning ...,"Developer, front-end",500 to 999 employees,Portugal,32127.0,11,HTML/CSS;JavaScript;Python;TypeScript
87903,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, back-end","1,000 to 4,999 employees",Portugal,32127.0,9,JavaScript;Python
88100,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, back-end",100 to 499 employees,Portugal,65967.0,8,Bash/Shell (all shells);HTML/CSS;Java;JavaScri...


In [59]:
# data_test=data_test[data_test['Country']!='United States of America']
data_test=data_test[data_test['Country']!='Other']
data_test

,RemoteWork,EdLevel,DevType,OrgSize,Country,ConvertedCompYearly,YearsCodePro,LanguageHaveWorkedWith
1,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Senior Executive (C-Suite, VP, etc.)",2 to 9 employees,United States of America,285000.0,9,HTML/CSS;JavaScript;Python
2,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, back-end","5,000 to 9,999 employees",United States of America,250000.0,23,Bash/Shell (all shells);Go
3,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, front-end",100 to 499 employees,United States of America,156000.0,7,Bash/Shell (all shells);HTML/CSS;JavaScript;PH...
5,Remote,Some college/university study without earning ...,"Developer, back-end",100 to 499 employees,United Kingdom of Great Britain and Northern I...,96828.0,21,Bash/Shell (all shells);HTML/CSS;JavaScript;Ru...
6,Remote,Some college/university study without earning ...,"Developer, full-stack","1,000 to 4,999 employees",United States of America,135000.0,3,Ada;Clojure;Elixir;Go;HTML/CSS;Java;JavaScript...
...,...,...,...,...,...,...,...,...
89160,"Hybrid (some remote, some in-person)","Professional degree (JD, MD, Ph.D, Ed.D, etc.)","Developer, mobile","1,000 to 4,999 employees",Mexico,28625.0,5,C;C#;C++;HTML/CSS;Java;Kotlin;MATLAB;Objective...
89162,Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Developer, back-end","5,000 to 9,999 employees",Brazil,50719.0,24,C;C++
89167,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, mobile",20 to 99 employees,India,15752.0,2,Dart;HTML/CSS;JavaScript;TypeScript
89170,"Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Other (please specify):,500 to 999 employees,France,64254.0,9,C#;F#;HTML/CSS;JavaScript;SQL;TypeScript


In [60]:
data_test['Country'].value_counts()

Country
United States of America                                10862
Germany                                                  3874
United Kingdom of Great Britain and Northern Ireland     3470
Canada                                                   2050
France                                                   1774
Netherlands                                              1338
Australia                                                1189
India                                                    1184
Poland                                                   1174
Spain                                                    1129
Sweden                                                   1014
Brazil                                                   1009
Italy                                                     992
Switzerland                                               634
Austria                                                   515
Denmark                                                   499


### Languages

In [61]:
series= data_test['LanguageHaveWorkedWith']

data_test_split = series.str.split(';')

# Explode the Series to get individual elements
exploded_series = data_test_split.explode()

# Get the unique values from the exploded Series
unique_values = exploded_series.unique()

print(unique_values)

['HTML/CSS' 'JavaScript' 'Python' 'Bash/Shell (all shells)' 'Go' 'PHP'
 'Ruby' 'SQL' 'TypeScript' 'Ada' 'Clojure' 'Elixir' 'Java' 'Lisp' 'OCaml'
 'Raku' 'Scala' 'Swift' 'Zig' 'Rust' 'C#' 'PowerShell' 'Kotlin' 'Solidity'
 'C' 'C++' 'Perl' 'Dart' 'Haskell' 'Delphi' 'Lua' 'VBA'
 'Visual Basic (.Net)' 'Groovy' 'GDScript' 'F#' 'R' 'Objective-C' 'APL'
 'Assembly' 'Cobol' 'Fortran' 'Julia' 'MATLAB' 'Prolog' 'Apex' nan
 'Crystal' 'SAS' 'Erlang' 'Nim' 'Flow']


In [62]:
language_list = unique_values
for i in language_list:
    data_test[i]=0
temporary_df = data['LanguageHaveWorkedWith'].str.get_dummies(';')

# Get the intersection of language_list and columns in temporary_df
columns_to_keep = list(set(language_list) & set(temporary_df.columns))

# Select the relevant columns and assign them to data_test
data_test[columns_to_keep] = temporary_df[columns_to_keep]


data_test.columns

Index([             'RemoteWork',                 'EdLevel',
                       'DevType',                 'OrgSize',
                       'Country',     'ConvertedCompYearly',
                  'YearsCodePro',  'LanguageHaveWorkedWith',
                      'HTML/CSS',              'JavaScript',
                        'Python', 'Bash/Shell (all shells)',
                            'Go',                     'PHP',
                          'Ruby',                     'SQL',
                    'TypeScript',                     'Ada',
                       'Clojure',                  'Elixir',
                          'Java',                    'Lisp',
                         'OCaml',                    'Raku',
                         'Scala',                   'Swift',
                           'Zig',                    'Rust',
                            'C#',              'PowerShell',
                        'Kotlin',                'Solidity',
                        

### Years of Exp

In [63]:
data_test.loc[data_test['YearsCodePro'] == 'More than 50 years', 'YearsCodePro'] = 50
data_test.loc[data_test['YearsCodePro'] == 'Less than 1 year', 'YearsCodePro'] = 0
data_test.loc[:, 'YearsCodePro'] = pd.to_numeric(data_test['YearsCodePro'])
# data_test['YearsCodePro'].value_counts()

In [64]:
for index, row in data_test.iterrows():
    if 6 <= row['YearsCodePro'] <= 10:
        data_test.loc[index, 'YearsCodePro'] = '6 to 10 years'
    elif 11 <= row['YearsCodePro'] <= 20:
        data_test.loc[index, 'YearsCodePro'] = '11 to 20 years'
    elif 21 <= row['YearsCodePro'] <= 30:
        data_test.loc[index, 'YearsCodePro'] = '21 to 30 years'
    elif 31 <= row['YearsCodePro'] :
        data_test.loc[index, 'YearsCodePro'] = 'More than 30 years'

data_test['YearsCodePro'].value_counts()

YearsCodePro
6 to 10 years         11012
11 to 20 years        10247
21 to 30 years         4154
5.0                    2954
3.0                    2333
4.0                    2312
2.0                    2244
More than 30 years     1512
1.0                    1211
0.0                     813
Name: count, dtype: int64

In [65]:
data_test['DevType'].value_counts()

DevType
Developer, full-stack                            13911
Developer, back-end                               7642
Developer, front-end                              2454
Developer, desktop or enterprise applications     2062
Other (please specify):                           1252
Developer, mobile                                 1193
Developer, embedded applications or devices       1124
Engineering manager                               1114
DevOps specialist                                  846
Data scientist or machine learning specialist      796
Engineer, data                                     757
Research & Development role                        716
Cloud infrastructure engineer                      662
Senior Executive (C-Suite, VP, etc.)               584
Academic researcher                                531
Developer, game or graphics                        394
Data or business analyst                           376
Developer, QA or test                              280
Sy

In [66]:
data_test

,RemoteWork,EdLevel,DevType,OrgSize,Country,ConvertedCompYearly,YearsCodePro,LanguageHaveWorkedWith,HTML/CSS,JavaScript,...,Julia,MATLAB,Prolog,Apex,NaN,Crystal,SAS,Erlang,Nim,Flow
1,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Senior Executive (C-Suite, VP, etc.)",2 to 9 employees,United States of America,285000.0,6 to 10 years,HTML/CSS;JavaScript;Python,1,1,...,0,0,0,0,0,0,0,0,0,0
2,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, back-end","5,000 to 9,999 employees",United States of America,250000.0,21 to 30 years,Bash/Shell (all shells);Go,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, front-end",100 to 499 employees,United States of America,156000.0,6 to 10 years,Bash/Shell (all shells);HTML/CSS;JavaScript;PH...,1,1,...,0,0,0,0,0,0,0,0,0,0
5,Remote,Some college/university study without earning ...,"Developer, back-end",100 to 499 employees,United Kingdom of Great Britain and Northern I...,96828.0,21 to 30 years,Bash/Shell (all shells);HTML/CSS;JavaScript;Ru...,1,1,...,0,0,0,0,0,0,0,0,0,0
6,Remote,Some college/university study without earning ...,"Developer, full-stack","1,000 to 4,999 employees",United States of America,135000.0,3.0,Ada;Clojure;Elixir;Go;HTML/CSS;Java;JavaScript...,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89160,"Hybrid (some remote, some in-person)","Professional degree (JD, MD, Ph.D, Ed.D, etc.)","Developer, mobile","1,000 to 4,999 employees",Mexico,28625.0,5.0,C;C#;C++;HTML/CSS;Java;Kotlin;MATLAB;Objective...,1,0,...,0,1,0,0,0,0,0,0,0,0
89162,Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Developer, back-end","5,000 to 9,999 employees",Brazil,50719.0,21 to 30 years,C;C++,0,0,...,0,0,0,0,0,0,0,0,0,0
89167,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, mobile",20 to 99 employees,India,15752.0,2.0,Dart;HTML/CSS;JavaScript;TypeScript,1,1,...,0,0,0,0,0,0,0,0,0,0
89170,"Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Other (please specify):,500 to 999 employees,France,64254.0,6 to 10 years,C#;F#;HTML/CSS;JavaScript;SQL;TypeScript,1,1,...,0,0,0,0,0,0,0,0,0,0


In [67]:
data_test.drop('LanguageHaveWorkedWith', axis=1, inplace=True)
data_test.to_csv('data_test.csv', index=False)

### Dumming

In [68]:
data_test_dummied = pd.get_dummies(data_test)

In [69]:
data_test_dummied.columns

Index([            'ConvertedCompYearly',                        'HTML/CSS',
                            'JavaScript',                          'Python',
               'Bash/Shell (all shells)',                              'Go',
                                   'PHP',                            'Ruby',
                                   'SQL',                      'TypeScript',
       ...
                      'YearsCodePro_0.0',                'YearsCodePro_1.0',
                      'YearsCodePro_2.0',                'YearsCodePro_3.0',
                      'YearsCodePro_4.0',                'YearsCodePro_5.0',
           'YearsCodePro_11 to 20 years',     'YearsCodePro_21 to 30 years',
            'YearsCodePro_6 to 10 years', 'YearsCodePro_More than 30 years'],
      dtype='object', length=151)

In [70]:
data_test_dummied.shape

(38935, 151)

# Lets start the test

## Making the Train Split

In [71]:
X = data_test_dummied.drop('ConvertedCompYearly', axis=1)
X.columns = X.columns.astype(str)
y = data_test['ConvertedCompYearly']
X_train, X_test, y_train, y_test = train_test_split(X, y)

## KNN

In [72]:
from sklearn.neighbors import KNeighborsRegressor

knn=KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train,y_train)
knn_score = knn.score(X_test,y_test)
knn_pred = knn.predict(X_test)


## XGBoost

In [73]:
xgb=xgboost.XGBRegressor(max_depth=35,n_estimators=5)
xgb.fit(X_train,y_train)
xgb_score = xgb.score(X_test,y_test)
xgb_pred = xgb.predict(X_test)

c:\Users\vi_to\Desktop\IRONHACK\Week_8\Project\.conda\lib\site-packages\xgboost\data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\vi_to\Desktop\IRONHACK\Week_8\Project\.conda\lib\site-packages\xgboost\data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
c:\Users\vi_to\Desktop\IRONHACK\Week_8\Project\.conda\lib\site-packages\xgboost\data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
c:\Users\vi_to\Desktop\IRONHACK\Week_8\Project\.conda\lib\site-packages\xgboost\data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstanc

## Random Forest

In [74]:
from sklearn.ensemble import RandomForestRegressor

forest=RandomForestRegressor()
forest.fit(X_train,y_train)
forest_score=forest.score(X_test,y_test)
print(forest.score(X_test,y_test))
forest_pred = forest.predict(X_test)

0.5597642055714834


## Linear Regression

In [75]:
# from sklearn.linear_model import LinearRegression

# lr = LinearRegression()
# lr.fit(X_train, y_train)
# lr_score = lr.score(X_test,y_test)
# lr_pred = lr.predict(X_test)

## Scores

In [76]:
print(f'''
            KNN Score: {knn_score},
            XGB Score: {xgb_score},
            Forest Score: {forest_score}''')


            KNN Score: 0.3043089943725862,
            XGB Score: 0.4065154149576419,
            Forest Score: 0.5597642055714834


In [77]:
# import joblib

# # Save the model using joblib
# joblib.dump(forest, 'forest.p')

# # Load the model using joblib


In [85]:
import pickle

pickle.dump(forest, open('forest_2.pkl', 'wb'))


            LR Score: -7.106729890858045e+21
            KNN Score: 0.13897749619450817
            XGB Score: 0.3114521831017649
            Forest Score: 0.40857521002019537

SVM Score: -0.00062509347823414,
LR Score: 0.0010353981622411856,
KNN Score: -0.0028332019384962326,
XGB Score: -0.0801337384471077               #xgboost.XGBRegressor(max_depth=5,n_estimators=50)
Forest Score: -0.01091935065766192          

SVM Score: -0.00062509347823414,
LR Score: 0.0010353981622411856,
KNN Score: -0.0028332019384962326,
XGB Score: -0.07867944443822927,            xgb=xgboost.XGBRegressor(max_depth=5,n_estimators=50)
Forest Score: -0.01091935065766192

SVM Score: -0.0005551799925724854,
            LR Score: 0.027536114541120593,
            KNN Score: -1.981352075257929,
            XGB Score: -0.3133251062730871,
            Forest Score: -0.09871983328399114

## Mean Squared Error

In [79]:
from sklearn.metrics import mean_squared_error

# lr_mse = np.sqrt(mean_squared_error(y_test, lr_pred))
knn_mse = np.sqrt(mean_squared_error(y_test, knn_pred))
xgb_mse = np.sqrt(mean_squared_error(y_test, xgb_pred))
forest_mse = np.sqrt(mean_squared_error(y_test, forest_pred))

print(f'''
            KNN MSE: {knn_mse},
            XGB MSE: {xgb_mse},
            Forest MSE: {forest_mse}''')


            KNN MSE: 46655.41974179267,
            XGB MSE: 43092.196178450926,
            Forest MSE: 37113.89905663282


SVM MSE: 737065796483.2642,\
LR MSE: 744692539445.2394,\
KNN MSE: 1432480258862.624,\
XGB MSE: 2749395762012.5312,\
Forest MSE: 1356465334701.4614      

SVM MSE: 1396667454935.641 \
LR MSE: 1394349749085.008, \
KNN MSE: 1399749521579.2988,\
XGB MSE: 1563822453137.1375,\
Forest MSE: 1411036127147.5947

# Lets Start the final part... Prepare the Streamlit Backend

In [102]:
adapted_data=pd.read_csv('adapted_data.csv')

In [90]:
data_test

,RemoteWork,EdLevel,DevType,OrgSize,Country,ConvertedCompYearly,YearsCodePro,HTML/CSS,JavaScript,Python,...,Julia,MATLAB,Prolog,Apex,NaN,Crystal,SAS,Erlang,Nim,Flow
1,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Senior Executive (C-Suite, VP, etc.)",2 to 9 employees,United States of America,285000.0,6 to 10 years,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, back-end","5,000 to 9,999 employees",United States of America,250000.0,21 to 30 years,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, front-end",100 to 499 employees,United States of America,156000.0,6 to 10 years,1,1,0,...,0,0,0,0,0,0,0,0,0,0
5,Remote,Some college/university study without earning ...,"Developer, back-end",100 to 499 employees,United Kingdom of Great Britain and Northern I...,96828.0,21 to 30 years,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,Remote,Some college/university study without earning ...,"Developer, full-stack","1,000 to 4,999 employees",United States of America,135000.0,3.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89160,"Hybrid (some remote, some in-person)","Professional degree (JD, MD, Ph.D, Ed.D, etc.)","Developer, mobile","1,000 to 4,999 employees",Mexico,28625.0,5.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
89162,Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Developer, back-end","5,000 to 9,999 employees",Brazil,50719.0,21 to 30 years,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89167,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Developer, mobile",20 to 99 employees,India,15752.0,2.0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
89170,"Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Other (please specify):,500 to 999 employees,France,64254.0,6 to 10 years,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
X_train

,HTML/CSS,JavaScript,Python,Bash/Shell (all shells),Go,PHP,Ruby,SQL,TypeScript,Ada,...,YearsCodePro_0.0,YearsCodePro_1.0,YearsCodePro_2.0,YearsCodePro_3.0,YearsCodePro_4.0,YearsCodePro_5.0,YearsCodePro_11 to 20 years,YearsCodePro_21 to 30 years,YearsCodePro_6 to 10 years,YearsCodePro_More than 30 years
73175,0,0,0,0,0,0,0,0,0,0,...,False,False,True,False,False,False,False,False,False,False
68756,0,0,0,1,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,False
27826,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
3125,1,1,0,0,0,0,0,1,0,0,...,False,False,False,True,False,False,False,False,False,False
26759,0,0,1,1,0,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41478,0,0,1,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
2044,0,1,1,1,1,0,1,1,0,0,...,False,False,False,False,False,False,False,False,True,False
39505,1,1,0,0,0,1,0,1,0,0,...,False,False,False,False,False,False,True,False,False,False
6545,1,1,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False


In [92]:
adapted_data

,Bash/Shell (all shells),HTML/CSS,JavaScript,Ruby,SQL,TypeScript,C#,PowerShell,Kotlin,Python,...,YearsCodePro_0.0,YearsCodePro_1.0,YearsCodePro_2.0,YearsCodePro_3.0,YearsCodePro_4.0,YearsCodePro_5.0,YearsCodePro_11 to 20 years,YearsCodePro_21 to 30 years,YearsCodePro_6 to 10 years,YearsCodePro_More than 30 years
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
forest.predict(adapted_data)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Unnamed: 0


In [87]:
adapted_data

,Bash/Shell (all shells),HTML/CSS,JavaScript,Ruby,SQL,TypeScript,C#,PowerShell,Kotlin,Python,...,YearsCodePro_0.0,YearsCodePro_1.0,YearsCodePro_2.0,YearsCodePro_3.0,YearsCodePro_4.0,YearsCodePro_5.0,YearsCodePro_11 to 20 years,YearsCodePro_21 to 30 years,YearsCodePro_6 to 10 years,YearsCodePro_More than 30 years
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import streamlit as st
import pandas as pd

# Sample DataFrame (replace this with your actual DataFrame)
unique_values_grouped = pd.read_csv('data_2.csv')

# Create a Streamlit app
st.title("Attribute Selector")

# List of attributes where multiple selections are allowed
multi_select_attributes = [
    'Bash/Shell ( all shells )', 'HTML/CSS', 'JavaScript', 'Ruby', 'SQL',
    'TypeScript', 'C#', 'PowerShell', 'Kotlin', 'Python', 'Java', 'Perl',
    'Dart', 'Go', 'Haskell', 'PHP', 'Delphi', 'C++', 'Clojure', 'Elixir',
    'Lua', 'Rust', 'C', 'Scala', 'GDScript', 'F#', 'Groovy', 'Lisp', 'Swift',
    'Objective-C', 'Visual Basic (.Net)', 'R', 'VBA', 'APL', 'Assembly',
    'Cobol', 'Fortran', 'Julia', 'MATLAB', 'Prolog', 'Crystal', 'SAS',
    'Apex', 'Solidity', 'Erlang', 'Raku', 'Nim', 'Zig', 'Ada', 'OCaml',
    'Flow'
]

# Create a flag to track if "Programming Language" has been displayed
programming_language_displayed = False

# Attributes to show outside the "Programming Language" multi-select box
attributes_to_show = ['RemoteWork', 'EdLevel', 'DevType', 'OrgSize', 'Country', 'YearsCodePro']

# Create a skeleton DataFrame based on the training data
skeleton_data = pd.DataFrame(columns=attributes_to_show + multi_select_attributes)

# Allow the user to select "Programming Language" and other attributes
selected_programming_languages = []
selected_attributes = {}

for column in attributes_to_show:
    with st.expander(f"Select {column}"):
        options = ['Select one:'] + [str(val) for val in unique_values_grouped[column].unique()]
        selected_values = [st.selectbox(f"Select {column}", options)]
        if selected_values[0] == 'Select one:':
            selected_values = []
        selected_attributes[column] = selected_values

# Allow the user to select "Programming Language"
with st.expander("Programming Language"):
    selected_programming_languages = st.multiselect("Select programming languages you're familiar with:",
                                                    multi_select_attributes)

# Define a function to adapt the skeleton DataFrame with user inputs
def adapt_skeleton_with_user_inputs(skeleton_data, selected_attributes, selected_programming_languages):
    # Ensure that selected_attributes contains only valid columns
    valid_columns = set(skeleton_data.columns)
    selected_attributes = {k: v for k, v in selected_attributes.items() if k in valid_columns}
    
    # Update the skeleton DataFrame with user inputs
    skeleton_data = skeleton_data.append({**selected_attributes, **{lang: 1 if lang in selected_programming_languages else 0 for lang in multi_select_attributes}}, ignore_index=True)
    
    return skeleton_data

# Create a button to apply user inputs and adapt the skeleton DataFrame
if st.button("Apply User Inputs"):
    skeleton_data = adapt_skeleton_with_user_inputs(skeleton_data, selected_attributes, selected_programming_languages)
    st.write("Skeleton DataFrame Adapted with User Inputs:")
    st.write(skeleton_data)

# Optionally, save the adapted skeleton DataFrame to a CSV file
if st.button("Save Adapted Data to CSV"):
    skeleton_data.to_csv('adapted_data.csv', index=False)


AttributeError: partially initialized module 'streamlit' has no attribute 'title' (most likely due to a circular import)

In [ ]:
import pandas as pd

# Sample training dataset with dummy-encoded columns
training_data = pd.read_csv('data_test_dummied.csv')

# Create a user input DataFrame with selected attributes and programming languages
user_input_data = pd.DataFrame({
    'RemoteWork': [1],
    'EdLevel': [2],
    'DevType_Back-end': [1],  # Dummy-encoded column
    'Country_USA': [0],      # Dummy-encoded column
    # Include other columns as needed
})

# Ensure that user_input_data has the same columns as training_data
for column in training_data.columns:
    if column not in user_input_data.columns:
        user_input_data[column] = 0  # Add missing columns with default values (e.g., all zeros)

# Concatenate the user input data with the training data
adapted_data = pd.concat([training_data, user_input_data], axis=0, ignore_index=True)

# Now, adapted_data matches the structure and format of the training data


C:\Users\vi_to\AppData\Local\Temp\ipykernel_2100\3928530973.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  user_input_data[column] = 0  # Add missing columns with default values (e.g., all zeros)
C:\Users\vi_to\AppData\Local\Temp\ipykernel_2100\3928530973.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  user_input_data[column] = 0  # Add missing columns with default values (e.g., all zeros)
C:\Users\vi_to\AppData\Local\Temp\ipykernel_2100\3928530973.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [86]:
X=X.to_csv('X.csv',index=False)

In [94]:
data_test=data_test.to_csv('data_test.csv',index=False)